# **Entregable semana 2**
**1. Extraer los datos**

El script obtiene los datos meteorológicos en formato JSON, directamente desde la API, tanto para:

-El clima actual (con valores como temperatura, humedad, viento, etc.)

-El pronóstico de los próximos cinco días, con datos cada 3 horas.

-Se revisó la estructura de los datos para asegurar que fueran útiles, consistentes y en el idioma deseado.

**2. Limpiar los datos**

En el procesamiento:

-Se eliminaron campos irrelevantes como coordenadas geográficas, código de zona horaria o identificadores internos.

-Se estandarizaron los valores numéricos, como la temperatura (a °C o °F) y la velocidad del viento (en m/s o mph).

-Se extrajeron solo los datos de mediodía en el pronóstico para evitar redundancia.

In [ ]:
!pip install requests
!pip install plotext
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone
import time

API_key = "f0f0c07c117721896152aeca0fbbe31a"

def obtener_clima_actual(ciudad):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={ciudad}&units=metric&appid={API_key}"
    res = requests.get(url)
    if res.status_code != 200:
        print("❌ Ciudad no encontrada o error en la petición.")
        return None
    data = res.json()
    temperatura = data["main"]["temp"]
    humedad = data["main"]["humidity"]
    presion = data["main"]["pressure"]
    return {
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "ciudad": ciudad,
        "temperatura": temperatura,
        "humedad": humedad,
        "presion": presion,
        "tipo": "actual"
    }

def obtener_pronostico(ciudad):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={ciudad}&units=metric&appid={API_key}"
    res = requests.get(url)
    if res.status_code != 200:
        print("❌ Ciudad no encontrada o error en la petición.")
        return []
    data = res.json()
    pronosticos = []

    # Queremos solo 1 pronóstico por día a las 18:00 hrs
    fechas_procesadas = set()
    for item in data["list"]:
        dt_txt = item["dt_txt"]  # formato 'YYYY-MM-DD HH:MM:SS'
        fecha_str, hora_str = dt_txt.split()
        if hora_str == "18:00:00" and fecha_str not in fechas_procesadas:
            fechas_procesadas.add(fecha_str)
            pronosticos.append({
                "fecha": fecha_str,
                "ciudad": ciudad,
                "temperatura": item["main"]["temp"],
                "humedad": item["main"]["humidity"],
                "presion": item["main"]["pressure"],
                "tipo": "pronostico"
            })
    return pronosticos

def guardar_csv(nombre_archivo, datos):
    campos = ["fecha", "ciudad", "temperatura", "humedad", "presion", "tipo"]
    try:
        with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo:
            writer = csv.DictWriter(archivo, fieldnames=campos)
            writer.writeheader()
            writer.writerows(datos)
        print(f"✅ Datos guardados en {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error al guardar CSV: {e}")

def guardar_json(nombre_archivo, datos):
    try:
        with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
            json.dump(datos, archivo, indent=4)
        print(f"✅ Datos guardados en {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error al guardar JSON: {e}")

def main():
    ciudad = input("🏙️ Introduce el nombre de la ciudad para obtener el clima: ").strip()
    if not ciudad:
        print("❌ Debes ingresar un nombre de ciudad válido.")
        return

    clima_actual = obtener_clima_actual(ciudad)
    if clima_actual is None:
        return

    pronostico = obtener_pronostico(ciudad)
    if not pronostico:
        print("⚠️ No se pudo obtener pronóstico, solo se guardará el clima actual.")
        datos_guardar = [clima_actual]
    else:
        datos_guardar = [clima_actual] + pronostico

    # Guardar en archivos fijos sin fecha ni hora en el nombre
    guardar_csv("clima_datos.csv", datos_guardar)
    guardar_json("clima_datos.json", datos_guardar)

    print("\n📊 Clima actual y pronóstico (solo temperaturas y humedad):")
    for dato in datos_guardar:
        tipo = "Actual" if dato["tipo"] == "actual" else "Pronóstico"
        print(f"{tipo} - {dato['fecha']} - {dato['ciudad']}: {dato['temperatura']}°C, Humedad: {dato['humedad']}%, Presión: {dato['presion']} hPa")

if __name__ == "__main__":
    main()


🏙️ Introduce el nombre de la ciudad para obtener el clima: Apodaca
✅ Datos guardados en clima_datos.csv
✅ Datos guardados en clima_datos.json

📊 Clima actual y pronóstico (solo temperaturas y humedad):
Actual - 2025-05-15 - Apodaca: 41.55°C, Humedad: 13%, Presión: 1004 hPa
Pronóstico - 2025-05-16 - Apodaca: 35.65°C, Humedad: 32%, Presión: 1006 hPa
Pronóstico - 2025-05-17 - Apodaca: 35.11°C, Humedad: 38%, Presión: 1007 hPa
Pronóstico - 2025-05-18 - Apodaca: 34.14°C, Humedad: 39%, Presión: 1007 hPa
Pronóstico - 2025-05-19 - Apodaca: 36.57°C, Humedad: 29%, Presión: 1004 hPa
Pronóstico - 2025-05-20 - Apodaca: 39.17°C, Humedad: 10%, Presión: 1006 hPa


**🗂️ Diseño de la Nueva Estructura de Datos**

Para facilitar el análisis del clima, se diseñó una estructura de datos clara y concisa basada en una lista de diccionarios, donde cada diccionario representa un registro meteorológico con los siguientes campos:

In [ ]:
{
    "fecha": "2025-05-15 12:00",
    "ciudad": "Madrid",
    "temperatura": "21.5°C",
    "humedad": "53%",
    "presion": "1012 hPa"
}

Este diseño permite:

Agrupar registros por ciudad o fecha.

Acceder fácilmente a variables clave como temperatura y humedad.

Exportar sin pérdida de formato a CSV y JSON.

# **📝 Documentación del Proceso**
🔹 Métodos de extracción de datos
Herramienta utilizada: requests

API: OpenWeather

Formato: JSON

Autenticación: API Key en la URL de la solicitud.

🔹 Técnicas de limpieza aplicadas
Eliminación de campos innecesarios como coordenadas, zona horaria, etc.

Conversión de unidades de temperatura (K → °C o °F).

Redondeo de valores flotantes a 2 decimales.

Uniformización de formatos de fecha y hora.

🔹 Validaciones y transformaciones:
Verificación del código de estado HTTP (200 OK).

Conversión de strings a float o int para análisis numérico.
